In [1]:
import util
import model
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import t
import tensorflow as tf 
from tensorflow.keras import datasets, layers, models, callbacks, regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import StratifiedKFold, cross_val_score

2024-07-11 22:45:03.276068: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2024-07-11 22:45:03.382609: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Specify number of epochs for the two models
epochs_drop = 50
epochs_l2 = 50

# Specify K for K-fold cross validaton
num_folds = 5
df = num_folds-1

kfold = StratifiedKFold(n_splits=num_folds)

# Specify batch size for training
batch_size = 20

# Specify confidence level ( < 1)
cf_lvl = 0.95

In [3]:
train_images, train_labels, test_images, test_labels, img_shape = util.get_images()

X = np.concatenate((train_images, test_images), axis=0)
y = np.concatenate((train_labels, test_labels), axis=0)

In [4]:
def create_model_drop(
    filter_nos=[32, 32, 32],
    filter_size=(3, 3),
    dense_layers=[64],
    input_shape=img_shape,
    patience=5,
    dropoutRate=0.1,
    output_length=100
):
    """
    Args:
        filter_nos (tuple): number of filters in each Conv2D layer
        filter_size (tuple): size of Conv2D filters. Filters are assumed to have uniform sizes across layers.
        input_shape (tuple): shape of the input of the first layer
        dense_layer (tuple): number of hidden units in each Dense layer except the output layer. Output layer is hardcoded to the number of classes.
        patience (integer): number of epochs of no improvement until early stop triggers.
        dropoutRate (double < 1): rate of value drop
        output_length (integer): number of classes for output
    """
    # Would include a dropout parameter, but wasn't absolutely sure where those would be, though I would assume closer to the start.

    model = models.Sequential()
    callback = callbacks.EarlyStopping(monitor='loss',patience=patience) # EarlyStopping 

    for i, filter_no in enumerate(filter_nos):
        if i == 0:
            # first Conv2D requires input_shape
            model.add(
                layers.Conv2D(
                    filter_no, filter_size, activation="relu", input_shape=input_shape, kernel_regularizer='l2'
                )
            )
        else:
            model.add(layers.Conv2D(filter_no, filter_size, activation="relu"))

        if i + 1 < len(filter_nos):
            model.add(
                layers.MaxPooling2D((2, 2))
            )  # add MaxPooling layer to all but the last convolutional stack
            model.add(
                layers.Dropout(rate=dropoutRate)
            )  

    model.add(layers.Flatten())

    for dense_layer in dense_layers:
        model.add(layers.Dense(dense_layer, activation="relu"))

    model.add(layers.Dense(output_length))  # output layer

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [5]:
def create_model_l2(
    filter_nos=[32, 32, 32],
    filter_size=(3, 3),
    dense_layers=[64],
    input_shape=img_shape,
    patience=5,
    l2param=0.1,
    output_length=100
):
    """
    Args:
        filter_nos (tuple): number of filters in each Conv2D layer
        filter_size (tuple): size of Conv2D filters. Filters are assumed to have uniform sizes across layers.
        input_shape (tuple): shape of the input of the first layer
        dense_layer (tuple): number of hidden units in each Dense layer except the output layer. Output layer is hardcoded to the number of classes.
        patience (integer): number of epochs of no improvement until early stop triggers.
        l2param (double < 1): param given to L2
        output_length (integer): number of classes given to output
    """
    # Would include a dropout parameter, but wasn't absolutely sure where those would be, though I would assume closer to the start.

    model = models.Sequential()
    regularizers.L2(l2=l2param) #L2
    callback = callbacks.EarlyStopping(monitor='loss',patience=patience) # EarlyStopping 

    for i, filter_no in enumerate(filter_nos):
        if i == 0:
            # first Conv2D requires input_shape
            model.add(
                layers.Conv2D(
                    filter_no, filter_size, activation="relu", input_shape=input_shape, kernel_regularizer='l2'
                )
            )
        else:
            model.add(layers.Conv2D(filter_no, filter_size, activation="relu", kernel_regularizer='l2'))

        if i + 1 < len(filter_nos):
            model.add(
                layers.MaxPooling2D((2, 2))
            )  # add MaxPooling layer to all but the last convolutional stack

    model.add(layers.Flatten())

    for dense_layer in dense_layers:
        model.add(layers.Dense(dense_layer, activation="relu"))

    model.add(layers.Dense(output_length))  # output layer

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [6]:
stopper = callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [7]:
print("Dropoff\n")

drop_acc = []
drop_loss = [] 

for train, test in kfold.split(X,y):
    model_drop = create_model_drop(
        filter_nos=(32, 64, 64),
        filter_size=(3,3),
        dense_layers=(64,),
        input_shape=img_shape,
        patience=5,
        dropoutRate=0.1,
        output_length=100
    )

    history_drop = model_drop.fit(
        X[train], y[train], epochs=epochs_drop, batch_size=batch_size, callbacks=[stopper], validation_data=(X[test], y[test])
    )

    test_loss, test_acc = model_drop.evaluate(X[test], y[test], verbose=2)
    drop_loss.append(test_loss)
    drop_acc.append(test_acc)

print("\n\ndrop model accuracy and loss:")
print(drop_acc)
print(drop_loss)

Dropoff



2024-07-11 22:45:11.764910: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-07-11 22:45:11.805798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.74GiB deviceMemoryBandwidth: 836.37GiB/s
2024-07-11 22:45:11.805824: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2024-07-11 22:45:11.848561: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-11 22:45:11.870557: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2024-07-11 22:45:11.877647: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-07-11 22:45:11.998100: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 22:45:12.012929: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2500000000 Hz
2024-07-11 22:45:12.013028: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560314799980 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-07-11 22:45:12.013036: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-07-11 22:45:12.014166: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
core

2378/2400 [============================>.] - ETA: 0s - loss: 4.0702 - accuracy: 0.0772WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x14a6668a2280> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2400/2400 [==============================] - 5s 2ms/step - loss: 4.0671 - accuracy: 0.0776 - val_loss: 3.6961 - val_accuracy: 0.131

In [8]:
print("L2\n")
#classifier_l2 = KerasClassifier(
#    create_model_l2,
#    filter_nos=(32, 64, 64),
#    filter_size=(3, 3),
#    dense_layers=(64,),
#    input_shape=image_shape,
#    patience=5,
#    l2param=0.01,
#    output_length=100,
#    epochs=epochs_l2,
#    batch_size=32,
#)

#l2Scores = dropScores = cross_validate(
#    estimator=classifier_l2, X=train_images, y=train_labels, cv=num_folds, scoring=make_scorer(accuracy_score)
#)

l2_acc = []
l2_loss = []

for train, test in kfold.split(X,y):
    model_l2 = create_model_l2(
        filter_nos=(32, 64, 64),
        filter_size=(3, 3),
        dense_layers=(64,),
        input_shape=img_shape,
        patience=5,
        l2param=1e-4,
        output_length=100
    )

    history_l2 = model_l2.fit(
        X[train], y[train], epochs=epochs_drop, batch_size=batch_size, callbacks=[stopper], validation_data=(X[test], y[test])
    )

    test_loss, test_acc = model_drop.evaluate(X[test], y[test], verbose=2)
    l2_loss.append(test_loss)
    l2_acc.append(test_acc)

print("\nl2 model accuracy and loss:")
print(l2_acc)
print(l2_loss)

L2

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2391/2400 [============================>.] - ETA: 0s - loss: 4.2337 - accuracy: 0.0489WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x14a5448648b0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH

In [9]:
diff = util.diff_scores(drop_acc, l2_acc)
mean_diff = np.mean(diff)
std_diff = np.std(diff, ddof=df)
cf_interval = t.interval(cf_lvl, df, loc=mean_diff, scale=std_diff/np.sqrt(num_folds))

In [10]:
print(f"\nConfidence Interval = {cf_interval}")


Confidence Interval = (-0.23115489084203783, 0.038154884166317626)
